# ideal

In [ ]:
import math
import pickle
import random
from typing import List, Tuple

import numpy as np
import torch
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm


class Solution:
    def __init__(self, n_estimators: int = 100, lr: float = 0.5, ndcg_top_k: int = 10,
                 subsample: float = 0.6, colsample_bytree: float = 0.9,
                 max_depth: int = 7, min_samples_leaf: int = 8):
        self._prepare_data()
        self.num_input_features = self.X_train.shape[1]
        self.num_train_objects = self.X_train.shape[0]
        self.num_test_objects = self.X_test.shape[0]

        self.features_to_choice = int(
            self.num_input_features * colsample_bytree)
        self.objects_to_choice = int(self.num_train_objects * subsample)

        self.ndcg_top_k = ndcg_top_k
        self.n_estimators = n_estimators
        self.lr = lr
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf

        self.trees = None
        self.trees_feat_idxs = None
        self.best_ndcg = -1
        self.best_iter_idx = -1

    def _get_data(self) -> List[np.array]:
        train_df, test_df = msrank_10k()

        X_train = train_df.drop([0, 1], axis=1).values
        y_train = train_df[0].values
        query_ids_train = train_df[1].values.astype(int)

        X_test = test_df.drop([0, 1], axis=1).values
        y_test = test_df[0].values
        query_ids_test = test_df[1].values.astype(int)

        return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

    def _prepare_data(self) -> None:
        (X_train, y_train, self.query_ids_train,
            X_test, y_test, self.query_ids_test) = self._get_data()

        X_train = self._scale_features_in_query_groups(
            X_train, self.query_ids_train)
        X_test = self._scale_features_in_query_groups(
            X_test, self.query_ids_train)

        self.X_train = torch.FloatTensor(X_train)
        self.X_test = torch.FloatTensor(X_test)

        self.ys_train = torch.FloatTensor(y_train).reshape(-1, 1)
        self.ys_test = torch.FloatTensor(y_test).reshape(-1, 1)

    def _scale_features_in_query_groups(self, inp_feat_array: np.array,
                                        inp_query_ids: np.array) -> np.array:
        for cur_id in np.unique(inp_query_ids):
            mask = inp_query_ids == cur_id
            tmp_array = inp_feat_array[mask]
            scaler = StandardScaler()
            inp_feat_array[mask] = scaler.fit_transform(tmp_array)

        return inp_feat_array

    def _train_one_tree(self, cur_tree_idx: int,
                        train_preds: torch.FloatTensor
                        ) -> Tuple[DecisionTreeRegressor, np.array]:
        lambdas = torch.zeros(self.num_train_objects, 1)
        for cur_id in np.unique(self.query_ids_train):
            train_mask = self.query_ids_train == cur_id
            lambda_update = self._compute_lambdas(
                self.ys_train[train_mask], train_preds[train_mask])
            if any(torch.isnan(lambda_update)):
                lambda_update = torch.zeros_like(lambda_update)
            lambdas[train_mask] = lambda_update

        tree = DecisionTreeRegressor(
            max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf, random_state=cur_tree_idx)

        this_tree_feats = np.random.choice(
            list(range(self.num_input_features)), self.features_to_choice, replace=False)
        this_tree_objs = np.random.choice(
            list(range(self.num_train_objects)), self.objects_to_choice, replace=False)

        tree.fit(
            self.X_train[this_tree_objs.reshape(-1)
                         ][:, this_tree_feats].numpy(),
            -lambdas[this_tree_objs.reshape(-1), :].numpy()
        )

        return tree, this_tree_feats

    def _calc_data_ndcg(self, queries_list: np.array,
                        true_labels: torch.FloatTensor, preds: torch.FloatTensor) -> float:
        ndcgs = []
        for cur_id in np.unique(queries_list):
            mask = queries_list == cur_id
            cur_ndcg = self._ndcg_k(
                true_labels[mask], preds[mask], self.ndcg_top_k)
            if np.isnan(cur_ndcg):
                ndcgs.append(0)
                continue
            ndcgs.append(cur_ndcg)
        return np.mean(ndcgs)

    def fit(self):
        np.random.seed(0)
        self.trees = []
        self.trees_feat_idxs = []
        self.best_ndcg = -1
        self.best_iter_idx = -1

        train_preds = torch.zeros(self.num_train_objects, 1)
        test_preds = torch.zeros(self.num_test_objects, 1)

        train_ndcgs, test_ndcgs = [], []

        p_bar = tqdm(range(self.n_estimators))
        for cur_tree_idx in p_bar:
            tree, this_tree_feats = self._train_one_tree(
                cur_tree_idx, train_preds)
            self.trees.append(tree)
            self.trees_feat_idxs.append(this_tree_feats)

            cur_tree_train_data = self.X_train[:, this_tree_feats].numpy()
            train_preds += self.lr * \
                torch.FloatTensor(tree.predict(
                    cur_tree_train_data)).reshape(-1, 1)
            train_ndcg = self._calc_data_ndcg(
                self.query_ids_train, self.ys_train, train_preds)

            cur_tree_test_data = self.X_test[:, this_tree_feats].numpy()
            test_preds += self.lr * \
                torch.FloatTensor(tree.predict(
                    cur_tree_test_data)).reshape(-1, 1)
            test_ndcg = self._calc_data_ndcg(
                self.query_ids_test, self.ys_test, test_preds)

            if self.best_ndcg < test_ndcg:
                self.best_ndcg = test_ndcg
                self.best_iter_idx = cur_tree_idx

            train_ndcgs.append(train_ndcg)
            test_ndcgs.append(test_ndcg)
            p_bar.set_description_str(
                f'Test nDCG@{self.ndcg_top_k}={round(test_ndcg, 5)}')

        cut_idx = self.best_iter_idx + 1
        self.trees = self.trees[:cut_idx]
        self.trees_feat_idxs = self.trees_feat_idxs[:cut_idx]

    def predict(self, data: torch.FloatTensor) -> torch.FloatTensor:
        preds = torch.zeros(data.shape[0], 1)
        for cur_tree_idx in range(len(self.trees)):
            tree = self.trees[cur_tree_idx]
            feat_idx = self.trees_feat_idxs[cur_tree_idx]
            tmp_preds = tree.predict(data[:, feat_idx].numpy())
            preds += self.lr * torch.FloatTensor(tmp_preds).reshape(-1, 1)

        return preds

    def _compute_ideal_dcg(self, ys_true: torch.FloatTensor) -> float:
        def dcg(ys_true, ys_pred):
            _, argsort = torch.sort(ys_pred, descending=True, dim=0)
            ys_true_sorted = ys_true[argsort]
            ret = 0
            for i, l in enumerate(ys_true_sorted, 1):
                ret += (2 ** l - 1) / np.log2(1 + i)
            return ret
        ideal_dcg = dcg(ys_true, ys_true)
        return ideal_dcg

    def _compute_lambdas(self, y_true, y_pred):
        # рассчитаем нормировку, IdealDCG
        ideal_dcg = self._compute_ideal_dcg(y_true)
        N = 1 / ideal_dcg

        # рассчитаем порядок документов согласно оценкам релевантности
        _, rank_order = torch.sort(y_true, descending=True, axis=0)
        rank_order += 1

        with torch.no_grad():
            # получаем все попарные разницы скоров в батче
            pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.t()))

            # поставим разметку для пар, 1 если первый документ релевантнее
            # -1 если второй документ релевантнее
            Sij = self._compute_labels_in_batch(y_true)
            # посчитаем изменение gain из-за перестановок
            gain_diff = self._compute_gain_diff(y_true)

            # посчитаем изменение знаменателей-дискаунтеров
            decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - \
                (1.0 / torch.log2(rank_order.t() + 1.0))
            # посчитаем непосредственное изменение nDCG
            delta_ndcg = torch.abs(N * gain_diff * decay_diff)
            # посчитаем лямбды
            lambda_update = (0.5 * (1 - Sij) - 1 /
                             pos_pairs_score_diff) * delta_ndcg
            lambda_update = torch.sum(lambda_update, dim=1, keepdim=True)

            return lambda_update

    def _compute_labels_in_batch(self, y_true):
        rel_diff = y_true - y_true.t()
        pos_pairs = (rel_diff > 0).type(torch.float32)
        neg_pairs = (rel_diff < 0).type(torch.float32)
        Sij = pos_pairs - neg_pairs
        return Sij

    def _compute_gain_diff(self, y_true):
        gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.t())
        return gain_diff

    def _ndcg_k(self, ys_true, ys_pred, ndcg_top_k) -> float:
        def dcg(ys_true, ys_pred):
            _, argsort = torch.sort(ys_pred, descending=True, dim=0)
            argsort = argsort[:ndcg_top_k]
            ys_true_sorted = ys_true[argsort]
            ret = 0
            for i, l in enumerate(ys_true_sorted, 1):
                ret += (2 ** l - 1) / math.log2(1 + i)
            return ret
        ideal_dcg = dcg(ys_true, ys_true)
        pred_dcg = dcg(ys_true, ys_pred)
        return (pred_dcg / ideal_dcg).item()

    def save_model(self, path: str):
        state = {
            'trees': self.trees,
            'trees_feat_idxs': self.trees_feat_idxs,
            'best_ndcg': self.best_ndcg,
            'lr': self.lr
        }
        f = open(path, 'wb')
        pickle.dump(state, f)

    def load_model(self, path: str):
        f = open(path, 'rb')
        state = pickle.load(f)
        self.trees = state['trees']
        self.trees_feat_idxs = state['trees_feat_idxs']
        self.best_ndcg = state['best_ndcg']
        self.lr = state['lr']


# me

In [343]:
import math
import pickle
import random
from typing import List, Tuple

import numpy as np
import torch
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm


class Solution:
    def __init__(self, n_estimators: int = 100, lr: float = 0.5, ndcg_top_k: int = 10,
                 subsample: float = 0.6, colsample_bytree: float = 0.9,
                 max_depth: int = 5, min_samples_leaf: int = 8):
        self._prepare_data()

        self.ndcg_top_k = ndcg_top_k
        self.n_estimators = n_estimators
        
        self.lr = lr
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        
        self.lr = 0.2
        self.max_depth = 13
        self.min_samples_leaf = 13
        self.subsample = 0.25
        self.colsample_bytree = 0.85
                
    def _get_data(self) -> List[np.ndarray]:
        train_df, test_df = msrank_10k()

        X_train = train_df.drop([0, 1], axis=1).values
        y_train = train_df[0].values
        query_ids_train = train_df[1].values.astype(int)

        X_test = test_df.drop([0, 1], axis=1).values
        y_test = test_df[0].values
        query_ids_test = test_df[1].values.astype(int)

        return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

    def _prepare_data(self) -> None:
        (X_train, y_train, self.query_ids_train,
            X_test, y_test, self.query_ids_test) = self._get_data()

        self.X_train = self._scale_features_in_query_groups(X_train, self.query_ids_train)
        self.X_test = self._scale_features_in_query_groups(X_test, self.query_ids_test)

        self.ys_train = torch.FloatTensor(y_train).reshape(-1,1)
        self.ys_test = torch.FloatTensor(y_test).reshape(-1,1)

    def _scale_features_in_query_groups(self, inp_feat_array: np.ndarray,
                                        inp_query_ids: np.ndarray) -> np.ndarray:
        
        total_inp_scaled = np.empty((0, inp_feat_array.shape[1])) 
        
        total_ids_list = []
        
        for for_v in np.unique(inp_query_ids):
            ids_list = [i for i, v in enumerate(inp_query_ids) if v==for_v]
            
            scaler = StandardScaler()
            
            group_inp_scaled = scaler.fit_transform(inp_feat_array[ids_list])
                        
            total_inp_scaled = np.concatenate([total_inp_scaled, 
                                               group_inp_scaled])
    
            total_ids_list.append(ids_list)
        
        total_ids_list = [i for sl in total_ids_list for i in sl]
            
        sort_ind = np.argsort(total_ids_list)
        
        total_inp_scaled = torch.FloatTensor(total_inp_scaled[sort_ind])
     
        return total_inp_scaled
    
    def _train_one_tree(self, cur_tree_idx: int,
                        train_preds: torch.FloatTensor
                        ) -> Tuple[DecisionTreeRegressor, np.ndarray]:
        
        torch.manual_seed(cur_tree_idx)
        np.random.seed(cur_tree_idx)
        
        total_ids_list = []
        
        total_lambda = np.empty((0, 1)) 
        
        for train_id_cat in np.unique(self.query_ids_train):
            
            idx_list = [i for i, v in enumerate(self.query_ids_train) if v==train_id_cat]

            batch_ys = self.ys_train[idx_list]
            batch_pred = train_preds[idx_list]
            
            if len(batch_ys) > 0:
                total_lambda = np.concatenate([total_lambda, 
                                               self._compute_lambdas(batch_ys, batch_pred)])

                total_ids_list.append(idx_list)
            
        total_ids_list = [i for sl in total_ids_list for i in sl]
            
        sort_ind = np.argsort(total_ids_list)
        
        total_lambda = total_lambda[sort_ind]
            
        rows_cnt = self.X_train.shape[0]
        subsample_size = round(self.subsample * rows_cnt) # or int() # можно вынести в init
        rows_indices = np.random.choice(np.arange(rows_cnt), subsample_size, replace=False) 
        
        cols_cnt = self.X_train.shape[1]
        colsample_size = round(self.colsample_bytree * cols_cnt)
        cols_indices = np.random.choice(np.arange(cols_cnt), colsample_size, replace=False)
                
        X_sub = self.X_train[rows_indices][:, cols_indices]
        l_sub = total_lambda[rows_indices]
        
        dt_reg = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
        dt_reg.fit(X_sub,l_sub)
                
        return dt_reg, cols_indices

    def _calc_data_ndcg(self, queries_list: np.ndarray,
                        true_labels: torch.FloatTensor, preds: torch.FloatTensor) -> float:
        
        ndcgs = []
        
        for query_cat in np.unique(queries_list):

            idx_list = [i for i, v in enumerate(queries_list) if v==query_cat]

            ys_cat = true_labels[idx_list]
            preds_cat = preds[idx_list]

            ndcg_score = self._ndcg_k(ys_cat, preds_cat, self.ndcg_top_k)
            
            ndcgs.append(ndcg_score)

        return np.mean(ndcgs)

    def fit(self):
        np.random.seed(0)
        train_preds = torch.zeros(len(self.X_train))
        test_preds = torch.zeros(len(self.X_test))
        self.trees = []
        best_ndcg = 0
        ndcg_val_list = []
        cols_indices_list = []
#         p_bar = tqdm(range(self.n_est))
#         p_bar.set_desctiption_str()
        for t in range(self.n_estimators):
            dt_reg, cols_indices = self._train_one_tree(t, train_preds)

            train_preds = train_preds - self.lr * dt_reg.predict(self.X_train[:,cols_indices])
        
            self.trees.append(dt_reg)
            
            test_preds = test_preds - self.lr * dt_reg.predict(self.X_test[:,cols_indices])
            
            curr_ndcg = self._calc_data_ndcg(self.query_ids_test, self.ys_test, test_preds)
            ndcg_val_list.append(curr_ndcg)
            if curr_ndcg > best_ndcg:
                best_ndcg = curr_ndcg
                
            cols_indices_list.append(cols_indices)
            
        if len(self.trees)>1:
            self.trees = self.trees[:ndcg_val_list.index(best_ndcg)]
            
        self.best_ndcg = best_ndcg
        
        self.ndcg_val_list = ndcg_val_list
        self.cols_indices_list = cols_indices_list
                  
    def predict(self, data: torch.FloatTensor) -> torch.FloatTensor:
        preds = torch.zeros(len(data))
        for ix, t in enumerate(self.trees):
            preds = preds - self.lr * t.predict(data[:, self.cols_indices_list[ix]])
        return preds

    def _compute_lambdas(self, y_true: torch.FloatTensor, y_pred: torch.FloatTensor) -> torch.FloatTensor:
        ideal_dcg = self.compute_ideal_dcg(y_true)

        N = 1 / ideal_dcg
        _, rank_order = torch.sort(y_true, descending=True, axis=0)
        rank_order += 1

        with torch.no_grad():
            pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.t()))
            Sij = self.compute_labels_in_batch(y_true)
            gain_diff = self.compute_gain_diff(y_true)
            decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - (1.0 / torch.log2(rank_order.t() + 1.0))
            delta_ndcg = torch.abs(N * gain_diff * decay_diff)
            lambda_update =  (0.5 * (1 - Sij) - 1 / pos_pairs_score_diff) * delta_ndcg
            lambda_update = torch.sum(lambda_update, dim=1, keepdim=True) # суммируем вдоль строчек

            return lambda_update

    def _ndcg_k(self, ys_true: torch.Tensor, ys_pred: torch.Tensor,
                ndcg_top_k: int) -> float:

        ys_ideal_true_sort, _ = torch.sort(ys_true, descending=True, dim=0)
    
        if ndcg_top_k:
            ys_ideal_true_sort = ys_ideal_true_sort[:ndcg_top_k]

        ideal_dcg = 0
        for ix, e in enumerate(ys_ideal_true_sort):
            ideal_dcg += self.compute_gain(float(e), gain_scheme="exp2")/math.log2(ix+2)
    
#         ideal_dcg = self.compute_ideal_dcg(ys_ideal_true_sort)
        
        dcg_ = self.dcg(ys_true, ys_pred, gain_scheme="exp2", top_k=ndcg_top_k)
                
        return dcg_/ideal_dcg
    
    def dcg(self, ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str, top_k: int) -> float:
        ys_pred_sort, sort_indices = torch.sort(ys_pred, descending=True, dim=0)
        ys_true_sort = ys_true[sort_indices] 
        
        if top_k:
            ys_true_sort = ys_true_sort[:top_k]
            
        dcg_ = 0
        for ix, e in enumerate(ys_true_sort):
            dcg_ += self.compute_gain(float(e), gain_scheme)/math.log2(ix+2)

        return dcg_
    
    def compute_ideal_dcg(self, ys_true: torch.Tensor, ndcg_scheme: str = 'exp2') -> float:
        ys_ideal_true_sort, _ = torch.sort(ys_true, descending=True, dim=0)

        ideal_dcg = 0
        for ix, e in enumerate(ys_ideal_true_sort):
            ideal_dcg += self.compute_gain(float(e), ndcg_scheme)/math.log2(ix+2)

        if ideal_dcg == 0:
            ideal_dcg = 1
            
        return ideal_dcg
    
    def compute_gain(self, y_value: float, gain_scheme: str) -> float:
        if gain_scheme=="const":
            return y_value
        elif gain_scheme=="exp2":
            # return 2**y_value - 1
            return pow(2, y_value) - 1
        else:
            return None
        
    def save_model(self, path: str):
        with open(path, 'wb') as f:
            pickle.dump([self.trees, self.cols_indices_list, self.lr], f)

    def load_model(self, path: str):
        with open(path, 'rb') as f:
            self.trees, self.cols_indices_list, self.lr = pickle.load(f)
            
    def compute_labels_in_batch(self, y_true):
        rel_diff = y_true - y_true.t()
        pos_pairs = (rel_diff > 0).type(torch.float32)
        neg_pairs = (rel_diff < 0).type(torch.float32)
        Sij = pos_pairs - neg_pairs
        return Sij

    def compute_gain_diff(self, y_true, gain_scheme = 'exp2'):
        if gain_scheme == "exp2":
            gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.t())
        elif gain_scheme == "diff":
            gain_diff = y_true - y_true.t()
        else:
            raise ValueError(f"{gain_scheme} method not supported")
        return gain_diff


In [344]:
s = Solution()

In [269]:
# s.lr = 0.5
# ndcg_top_k = 10,
# s.subsample = 0.9
# colsample_bytree = 0.9,
# max_depth = 5, 
# min_samples_leaf = 8

In [345]:
s.fit()

In [346]:
s.best_ndcg

0.43199949666272697

In [347]:
s.save_model('output/model.bin')

In [348]:
s.load_model('output/model.bin')

In [349]:
test_preds = s.predict(s.X_test)

In [350]:
s._calc_data_ndcg(s.query_ids_test, s.ys_test, test_preds)

0.4318346664243743

In [331]:
import hyperopt

In [332]:
from hyperopt import STATUS_OK, Trials, fmin, hp, rand, tpe, space_eval

In [333]:
#зададим пространство поиска
space = [hp.choice('lr', np.arange(0.05, 1, 0.05)), 
         hp.choice('subsample', np.arange(0.1, 1, 0.05)),
         hp.choice('colsample_bytree', np.arange(0.1, 1, 0.05)),
         hp.choice('max_depth', np.arange(3, 15, 2)),
         hp.choice('min_samples_leaf', np.arange(3, 15, 2))
        ]

#укажем objective-функцию
def f(args):
    s.lr, s.subsample, s.colsample_bytree, s.max_depth, s.min_samples_leaf = args
    s.fit()
    test_preds = s.predict(s.X_test)
    ndcg_test = s._calc_data_ndcg(s.query_ids_test, s.ys_test, test_preds)
    return -ndcg_test

trials = Trials()

best = fmin(f, space, algo = tpe.suggest, max_evals=150, trials=trials)
print ('TPE result: ', best)

 55%|█████▍    | 82/150 [1:39:52<1:22:49, 73.08s/trial, best loss: -0.4318346664243743] 


KeyboardInterrupt: 

In [334]:
dict(zip(['lr','subsample', 'colsample_bytree', 'max_depth','min_samples_leaf'], 
         hyperopt.space_eval(space, trials.argmin)))

{'lr': 0.2,
 'subsample': 0.25000000000000006,
 'colsample_bytree': 0.8500000000000002,
 'max_depth': 13,
 'min_samples_leaf': 13}

In [337]:
best_trials = sorted(trials.trials, key=lambda x: x['result']['loss'], reverse=False)[20:40]

KeyError: 'loss'

In [338]:
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': -0.36977544952521574, 'status': 'ok'},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'colsample_bytree': [0],
    'lr': [0],
    'max_depth': [0],
    'min_samples_leaf': [0],
    'subsample': [0]},
   'vals': {'colsample_bytree': [3],
    'lr': [14],
    'max_depth': [1],
    'min_samples_leaf': [1],
    'subsample': [11]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2021, 7, 14, 19, 45, 12, 842000),
  'refresh_time': datetime.datetime(2021, 7, 14, 19, 46, 7, 90000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': -0.38172287350624196, 'status': 'ok'},
  'misc': {'tid': 1,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'colsample_bytree': [1],
    'lr': [1],
    'max_depth': [1],
    'min_samples_leaf': [1],
    'subsample': [1]},
   'vals': {'colsample_bytree': [3],
    

In [308]:
# best_trials

In [309]:
top_res_list = []
for bt in best_trials:
    d = bt['misc']['vals']
    d = {k:v[0] for k,v in d.items()}
    
    ddd = dict(zip(['lr','subsample', 'colsample_bytree', 'max_depth','min_samples_leaf'], 
             hyperopt.space_eval(space, d)))
    ddd['score'] = bt['result']['loss']
    
    top_res_list.append(ddd)

In [310]:
top_res_list

[{'lr': 0.45,
  'subsample': 0.3500000000000001,
  'colsample_bytree': 0.9500000000000003,
  'max_depth': 13,
  'min_samples_leaf': 7,
  'score': -0.4319659896309627},
 {'lr': 0.45,
  'subsample': 0.3500000000000001,
  'colsample_bytree': 0.9500000000000003,
  'max_depth': 13,
  'min_samples_leaf': 7,
  'score': -0.4319659896309627},
 {'lr': 0.15000000000000002,
  'subsample': 0.3500000000000001,
  'colsample_bytree': 0.9500000000000003,
  'max_depth': 13,
  'min_samples_leaf': 7,
  'score': -0.4319659896309627},
 {'lr': 0.45,
  'subsample': 0.3500000000000001,
  'colsample_bytree': 0.9500000000000003,
  'max_depth': 13,
  'min_samples_leaf': 7,
  'score': -0.4319659896309627},
 {'lr': 0.7000000000000001,
  'subsample': 0.3500000000000001,
  'colsample_bytree': 0.9500000000000003,
  'max_depth': 13,
  'min_samples_leaf': 7,
  'score': -0.4319659896309627},
 {'lr': 0.45,
  'subsample': 0.3500000000000001,
  'colsample_bytree': 0.9500000000000003,
  'max_depth': 13,
  'min_samples_leaf':